In [1]:
import time
import pandas as pd
import os

In [1]:
#test_dir = "C:\\Users\\soup\\Desktop\\DS400-Capstone\\DS400-Capstone\\main\\test.pcap"
#test_dir2 = "C:\\Users\\soup\\Desktop\\DS400-Capstone\\DS400-Capstone\\main\\test2.pcap"
#normal_dir = "D:\\Notebooks\\DS400-Capstone\\main\\Network_dataset_pcaps\\normal_pcaps"
#mal_dir = "D:\\Notebooks\\DS400-Capstone\\main\\Network_dataset_pcaps\\normal_attack_pcaps"

normal_dir = "D:\\Notebooks\\Datasets\\Network_dataset_pcaps\\normal_pcaps"
mal_dir = "D:\\Notebooks\\Datasets\\Network_dataset_pcaps\\normal_attack_pcaps"

In [3]:
os.listdir(normal_dir)

['normal_1.pcap',
 'normal_10.pcap',
 'normal_11.pcap',
 'normal_12.pcap',
 'normal_13.pcap',
 'normal_2.pcap',
 'normal_3.pcap',
 'normal_4.pcap',
 'normal_5.pcap',
 'normal_6.pcap',
 'normal_7.pcap',
 'normal_8.pcap',
 'normal_9.pcap',
 'normal_IoT_2.pcap',
 'normal_IoT_3.pcap']

In [4]:
os.listdir(mal_dir)

['Injection_normal',
 'MITM_normal',
 'normal_backdoor',
 'normal_DDoS',
 'normal_DoS',
 'normal_runsomware',
 'normal_scanning',
 'normal_XSS',
 'password_normal']

In [19]:
with open(normal_dir+"\\normal_1.pcap", "rb") as f:
        #Find magic number
        while (byte := f.read(1000)):
            print(byte.hex(sep=" "))
            break

0a 0d 0d 0a 34 00 00 00 4d 3c 2b 1a 01 00 00 00 ff ff ff ff ff ff ff ff 04 00 0d 00 45 64 69 74 63 61 70 20 32 2e 36 2e 35 00 00 00 00 00 00 00 34 00 00 00 01 00 00 00 14 00 00 00 71 00 00 00 00 00 04 00 14 00 00 00 06 00 00 00 48 02 00 00 00 00 00 00 8e 85 05 00 a5 fb 5b 1e 27 02 00 00 27 02 00 00 00 04 00 01 00 06 a4 91 b1 1e 58 a2 00 00 00 03 09 00 02 17 04 d2 00 00 7f 06 ed 8c c0 a8 01 31 c0 a8 01 01 09 00 00 00 00 01 f7 fa 00 00 00 00 50 12 04 00 5c 9c 00 00 09 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f 10 11 12 13 14 15 16 17 18 19 1a 1b 1c 1d 1e 1f 20 21 22 23 24 25 26 27 28 29 2a 2b 2c 2d 2e 2f 30 31 32 33 34 35 36 37 38 39 3a 3b 3c 3d 3e 3f 40 41 42 43 44 45 46 47 48 49 4a 4b 4c 4d 4e 4f 50 51 52 53 54 55 56 57 58 59 5a 5b 5c 5d 5e 5f 60 61 62 63 64 65 66 67 68 69 6a 6b 6c 6d 6e 6f 70 71 72 73 74 75 76 77 78 79 7a 7b 7c 7d 7e 7f 80 81 82 83 84 85 86 87 88 89 8a 8b 8c 8d 8e 8f 90 91 92 93 94 95 96 97 98 99 9a 9b 9c 9d 9e 9f a0 a1 a2 a3 a4 a5 a6 a7 a8 a9 aa ab ac ad ae af b0 b

In [6]:
# parse_pcap function needs to be able to handle v1.0 and v2.4 of pcap files

In [27]:
def parse_v1(f, df, label):
    #block_type = f.read(4)
    block_length = f.read(4)
    magic_seq = f.read(4)
    major_ver = f.read(2)
    minor_ver = f.read(2)
    sec_length = f.read(8)
    options = f.read(int.from_bytes(block_length, "little")-28)
    block_length2 = f.read(4)
    
    # Read in interface description block(s)
    b_type = f.read(4)      
    while int.from_bytes(b_type, "little") == 1:

        #i_block_type = f.read(4)
        i_block_length = f.read(4)
        link_type = f.read(2)
        res1 = f.read(2)
        snap_length = f.read(4)
        i_options = f.read(int.from_bytes(i_block_length, "little")-20)
        i_block_length2 = f.read(4)

        b_type = f.read(4)

    f.seek(-4, 1)

    test = []
    last_loc = 0
    # Loop through all packets
    while f.tell() != last_loc:
        last_loc = f.tell()
        # Read in packet header block
        p_block_type = f.read(4)
        p_block_length = int.from_bytes(f.read(4), "little")
        if p_block_length==0:
            break
        interfaceID = f.read(4)
        timestamp1 = f.read(4)
        timestamp2 = f.read(4)
        if p_block_length < 24:
            print("Block length too short?")
            print(p_block_length)
            print(len(test))
            break
        data = f.read(p_block_length-24)
        f.read(4)

        test.append(data)
        
    temp = pd.DataFrame(data=test, columns=["p_bytes"])
    temp["label"] = label
    new_df = pd.concat([df, temp], axis=0)
    new_df.reset_index(drop=True, inplace=True)
    return new_df
    

def parse_v2(f, df, label):
    
    #Read in file header
    version = f.read(4)
    reserves = f.read(8)
    snap_len = f.read(4)
    link_type = f.read(4)
    
    test = []
    last_loc = 0
    while f.tell() != last_loc:
        #Read in packet header
        last_loc = f.tell()
        ts1 = f.read(4)
        ts2 = f.read(4)
        cap_len = f.read(4)
        if int.from_bytes(cap_len, "little") == 0:
            break
        p_len = f.read(4)
        data = f.read(int.from_bytes(cap_len, "little"))
        test.append(data)
        
    temp = pd.DataFrame(data=test, columns=["p_bytes"])
    temp["label"] = label
    new_df = pd.concat([df, temp], axis=0)
    new_df.reset_index(drop=True, inplace=True)
    return new_df
    

def parse_pcap(file_dir, df, label):
    with open(file_dir, "rb") as f:
        
        magic_nums = ["d4c3b2a1"]
        starter = "0a0d0d0a"
        
        first_read = f.read(4)
        #print(first_read.hex())
        if first_read.hex() in magic_nums:
            print("Using v2.4")
            magic_seq = first_read
            return parse_v2(f, df , label)
        elif first_read.hex()==starter:
            print("Using v1.0")
            block_type = first_read
            return parse_v1(f, df, label)
        else:
            print("Unable to detect the pcap file format.")

In [8]:
df = pd.DataFrame(columns=["p_bytes"])

In [9]:
temp = pd.DataFrame(columns=["p_bytes"])
temp = parse_pcap(normal_dir+"\\normal_1.pcap", temp, 0)

0a0d0d0a
Using v1.0


In [11]:
temp['p_bytes'][0].hex(sep=" ")

'2702000027020000000400010006a491b11e58a2000000030900021704d200007f06ed8cc0a80131c0a80101090000000001f7fa00000000501204005c9c0000090102030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f303132333435363738393a3b3c3d3e3f404142434445464748494a4b4c4d4e4f505152535455565758595a5b5c5d5e5f606162636465666768696a6b6c6d6e6f707172737475767778797a7b7c7d7e7f808182838485868788898a8b8c8d8e8f909192939495969798999a9b9c9d9e9fa0a1a2a3a4a5a6a7a8a9aaabacadaeafb0b1b2b3b4b5b6b7b8b9babbbcbdbebfc0c1c2c3c4c5c6c7c8c9cacbcccdcecfd0d1d2d3d4d5d6d7d8d9dadbdcdddedfe0e1e2e3e4e5e6e7e8e9eaebecedeeeff0f1f2f3f4f5f6f7f8f9fafbfcfdfeff000102030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f303132333435363738393a3b3c3d3e3f404142434445464748494a4b4c4d4e4f505152535455565758595a5b5c5d5e5f606162636465666768696a6b6c6d6e6f707172737475767778797a7b7c7d7e7f808182838485868788898a8b8c8d8e8f909192939495969798999a9b9c9d9e9fa0a1a2a3a4a5a6a7a8a9aaabacadaeafb0b1b2b

In [12]:
temp = pd.DataFrame(columns=["p_bytes"])
temp = parse_pcap(mal_dir+"\\normal_runsomware\\normal_runsomware2.pcap", temp, 1)

0a0d0d0a
Using v1.0


In [16]:
temp['p_bytes'][0].hex(sep=" ")

'36 01 00 00 36 01 00 00 00 0c 29 b8 b1 74 00 0c 29 a4 03 f4 08 00 45 00 01 28 06 69 40 00 80 06 6f 34 c0 a8 01 c1 c0 a8 01 21 c0 30 11 5c 1d dc ca bf 1b 8e ea 32 50 18 3f 51 47 ad 00 00 2a b4 ef 55 d0 93 aa 48 82 a5 a9 93 9f 79 33 74 ba 3b 46 94 2a b4 ef 54 2a b4 ef bd 2a b4 ef 54 ac 14 9a 71 3d d6 31 81 3a 2e 7d c1 ff 58 c5 6e 2a 95 5e 74 7e f0 39 b6 56 0a 45 1a 50 32 00 c7 ac 09 2f bd 0f 59 f5 08 0e 75 9d 86 2f 13 9d 64 8d f5 14 66 8f 8c fd 46 d1 88 31 50 f9 83 f1 2d fd 42 fd d4 a5 0f 76 97 39 4e b3 e6 0e af f4 9e 5d f1 ed 0f d4 f3 d9 aa b7 da 56 0d 20 7d 5f 6b 25 9d a9 3c 67 d7 7c ce 24 61 cc 5e 46 11 33 49 9c 0a 97 32 c8 03 a7 50 1b 4f 08 66 c4 a5 ae 84 55 9e 25 05 e6 8a 4f 18 df e3 2e 63 32 fa 0d b2 52 8e 62 d3 10 e7 4c 75 84 a8 9b 1f 3b 79 80 ed c5 96 7e 92 e5 69 8c 3c 1f a0 ac c8 59 07 40 1e 1b 1c 3b 1d bd 99 ac ea 38 d9 2a 12 c4 64 ef b8 a1 ec f8 c8 30 c0 70 dc 38 05 e4 a3 a9 a8 20 b6 c2 c5 51 13 19 da e1 6a cd 1b 5a 5f a6 8f 63 09 00 00'

In [17]:
temp = pd.DataFrame(columns=["p_bytes"])
temp = parse_pcap("D:\\Notebooks\\DS400-Capstone\\main\\Network_dataset_pcaps\\normal_attack_pcaps\\normal_XSS\\normal_XSS1.pcap", temp, 1)

d4c3b2a1
Using v2.4


In [18]:
temp['p_bytes'][0].hex(sep=" ")

'00 0c 29 5b 73 5b 00 0c 29 28 72 d1 08 00 45 00 00 34 46 06 40 00 40 06 70 b1 c0 a8 01 98 c0 a8 01 24 00 50 93 18 72 f3 26 81 db 64 22 37 80 10 00 f9 3a 13 00 00 01 01 08 0a 92 ce c1 8b 38 0e 00 c3'

In [20]:
# Parse all normal pcaps and insert them into df
i = 1
for file_name in os.listdir(normal_dir):
    print(str(i)+"/"+str(len(os.listdir(normal_dir))), end=" : ")
    df = parse_pcap(normal_dir+"\\"+file_name, df, 0)
    i += 1

0a0d0d0a
Using v1.0
0a0d0d0a
Using v1.0
0a0d0d0a
Using v1.0
0a0d0d0a
Using v1.0
0a0d0d0a
Using v1.0
0a0d0d0a
Using v1.0
0a0d0d0a
Using v1.0
0a0d0d0a
Using v1.0
0a0d0d0a
Using v1.0
0a0d0d0a
Using v1.0
0a0d0d0a
Using v1.0
0a0d0d0a
Using v1.0
0a0d0d0a
Using v1.0
d4c3b2a1
Using v2.4
d4c3b2a1
Using v2.4


In [21]:
len(df)

31287179

In [22]:
df.to_pickle('./raw_normal.pkl')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31287179 entries, 0 to 31287178
Data columns (total 2 columns):
 #   Column   Dtype  
---  ------   -----  
 0   p_bytes  object 
 1   label    float64
dtypes: float64(1), object(1)
memory usage: 477.4+ MB


In [24]:
df.head()

,p_bytes,label
0,b'\'\x02\x00\x00\'\x02\x00\x00\x00\x04\x00\x01...,0.0
1,b'(\x02\x00\x00(\x02\x00\x00\x00\x04\x00\x01\x...,0.0
2,b'\xbd\x00\x00\x00\xbd\x00\x00\x00\x00\x03\x00...,0.0
3,b'\xbd\x00\x00\x00\xbd\x00\x00\x00\x00\x00\x00...,0.0
4,b'D\x00\x00\x00D\x00\x00\x00\x00\x04\x00\x01\x...,0.0


In [31]:
del(df)

In [30]:
# Read in all malicious data to dataframe
df = pd.DataFrame(columns=["p_bytes"])

for folder in os.listdir(mal_dir):
    i = 1
    print(folder)
    for file_name in os.listdir(mal_dir+"\\"+folder):    
        print(str(i)+"/"+str(len(os.listdir(mal_dir+"\\"+folder))), end=" : ")
        df = parse_pcap(mal_dir+"\\"+folder+"\\"+file_name, df, 1)
        i += 1

Injection_normal
1/4 : Using v2.4
2/4 : Using v1.0
3/4 : Using v1.0
4/4 : Using v2.4
MITM_normal
1/4 : Using v2.4
2/4 : Using v2.4
3/4 : Using v2.4
4/4 : Using v2.4
normal_backdoor
1/1 : Using v1.0
normal_DDoS
1/12 : Using v2.4
2/12 : Using v1.0
3/12 : Using v1.0
4/12 : Using v1.0
5/12 : Using v2.4
6/12 : Using v1.0
7/12 : Using v1.0
8/12 : Using v1.0
9/12 : Using v1.0
10/12 : Using v1.0
11/12 : Using v1.0
12/12 : Using v1.0
normal_DoS
1/5 : Using v2.4
2/5 : Using v1.0
3/5 : Using v1.0
4/5 : Using v1.0
5/5 : Using v1.0
normal_runsomware
1/2 : Using v1.0
2/2 : Using v1.0
normal_scanning
1/6 : Using v2.4
2/6 : Using v2.4


KeyboardInterrupt: 

In [ ]:
len(df)

In [ ]:
df.to_pickle('./raw_mal.pkl')

In [110]:
with open(test_dir, "rb") as f:
        
        # Read in section header block
        block_type = f.read(4)
        block_length = f.read(4)
        magic_seq = f.read(4)
        major_ver = f.read(2)
        minor_ver = f.read(2)
        sec_length = f.read(8)
        options = f.read(int.from_bytes(block_length, "little")-28)
        block_length2 = f.read(4)
        
        # Read in interface description block
        b_type = f.read(4)
        
        while int.from_bytes(b_type, "little") == 1:
        
            #i_block_type = f.read(4)
            i_block_length = f.read(4)
            link_type = f.read(2)
            res1 = f.read(2)
            snap_length = f.read(4)
            i_options = f.read(int.from_bytes(i_block_length, "little")-20)
            i_block_length2 = f.read(4)
            
            b_type = f.read(4)
        
        # TODO: HAVE TO CHECK IF NEXT BLOCK IS ANOTHER INTERFACE BLOCK OR A PACKET BLOCK
        
        f.seek(-4)
        
        test = []
        last_loc = 0
        # Loop through all packets
        while f.tell() != last_loc:
            last_loc = f.tell()
            # Read in packet header block
            p_block_type = f.read(4)
            p_block_length = int.from_bytes(f.read(4), "little")
            if p_block_length==0:
                break
            interfaceID = f.read(4)
            timestamp1 = f.read(4)
            timestamp2 = f.read(4)
            cap_len = int.from_bytes(f.read(4), "little")
            p_len = f.read(4)

            # Read in packet data
            data = f.read(cap_len)
            padding = 4-(cap_len%4)
            if padding==4:
                padding=0
            f.read(padding)
            options = f.read( p_block_length-(32+cap_len+padding) )
            p_block_length2 = f.read(4)
            
            test.append(cap_len)
        

In [86]:
p_block_length.hex(sep=" ")

'48 02 00 00'

In [13]:
int.from_bytes(block_length, "little")

52

In [14]:
df.head()

,p_bytes,label
0,b'\'\x02\x00\x00\'\x02\x00\x00\x00\x04\x00\x01...,0.0
1,b'(\x02\x00\x00(\x02\x00\x00\x00\x04\x00\x01\x...,0.0
2,b'\xbd\x00\x00\x00\xbd\x00\x00\x00\x00\x03\x00...,0.0
3,b'\xbd\x00\x00\x00\xbd\x00\x00\x00\x00\x00\x00...,0.0
4,b'D\x00\x00\x00D\x00\x00\x00\x00\x04\x00\x01\x...,0.0


In [307]:
test = []
for packet in df['p_bytes']:
    #test.append([x+1 for x in list(packet)])
    test.append(list(packet))

In [15]:
len(test)

NameError: name 'test' is not defined

In [309]:
df['tokenized'] = test

In [311]:
df.head()

,p_bytes,label,tokenized
0,b'\'\x02\x00\x00\'\x02\x00\x00\x00\x04\x00\x01...,0.0,"[39, 2, 0, 0, 39, 2, 0, 0, 0, 4, 0, 1, 0, 6, 1..."
1,b'(\x02\x00\x00(\x02\x00\x00\x00\x04\x00\x01\x...,0.0,"[40, 2, 0, 0, 40, 2, 0, 0, 0, 4, 0, 1, 0, 6, 1..."
2,b'\xbd\x00\x00\x00\xbd\x00\x00\x00\x00\x03\x00...,0.0,"[189, 0, 0, 0, 189, 0, 0, 0, 0, 3, 0, 1, 0, 6,..."
3,b'\xbd\x00\x00\x00\xbd\x00\x00\x00\x00\x00\x00...,0.0,"[189, 0, 0, 0, 189, 0, 0, 0, 0, 0, 0, 1, 0, 6,..."
4,b'D\x00\x00\x00D\x00\x00\x00\x00\x04\x00\x01\x...,0.0,"[68, 0, 0, 0, 68, 0, 0, 0, 0, 4, 0, 1, 0, 6, 0..."


In [ ]:
# Create neural network utilizing embedding layer